In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [3]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-09
DEST_DIR: ..\data

source_path: ..\data\2025-04-09_df_OHLCV_clean.parquet
dest_path: ..\data\2025-04-09_df_perf_ratios.parquet


In [4]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [5]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open    High     Low   Close  Adj Close   Volume    Adj Open    Adj High     Adj Low
Symbol Date                                                                                              
INVH   2025-04-09   29.91   32.36   29.37   32.27      32.27  4914785   29.910000   32.360000   29.370000
       2025-04-08   31.96   32.16   30.10   30.53      30.53  3600600   31.960000   32.160000   30.100000
       2025-04-07   32.67   32.94   30.96   31.44      31.44  5098400   32.670000   32.940000   30.960000
       2025-04-04   33.78   34.13   32.44   32.54      32.54  4671900   33.780000   34.130000   32.440000
       2025-04-03   34.35   34.72   33.88   33.99      33.99  2766900   34.350000   34.720000   33.880000
...                   ...     ...     ...     ...        ...      ...         ...         ...         ...
GD     2024-04-16  285.46  286.38  283.68  285.30     281.02  1388500  281.177600  282.083798  279.424303
       2024-04-15  291.71  291.75  284.36  284.75     280.48   976600  287.335631  287.375031  280.095848
       2024-04-12  288.41  290.75  286.79  287.87     283.56  1280300  284.091915  286.396881  282.496170
       2024-04-11  287.83  288.64  286.16  287.50     283.19   710900  283.515053  284.312910  281.870088
       2024-04-10  287.40  290.74  286.66  290.04     284.29   935600  281.702338  284.976123  280.977008

[385536 rows x 9 columns]

In [6]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
INVH   2025-04-09      32.27
       2025-04-08      30.53
       2025-04-07      31.44
       2025-04-04      32.54
       2025-04-03      33.99
...                      ...
GD     2024-04-16     281.02
       2024-04-15     280.48
       2024-04-12     283.56
       2024-04-11     283.19
       2024-04-10     284.29

[385536 rows x 1 columns]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 385536 entries, ('INVH', Timestamp('2025-04-09 00:00:00')) to ('GD', Timestamp('2024-04-10 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  385536 non-null  float64
dtypes: float64(1)
memory usage: 4.5+ MB


In [8]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-09     198.85
       2025-04-08     172.42
       2025-04-07     181.46
       2025-04-04     188.38
       2025-04-03     203.19
...                      ...
IYR    2024-04-16      80.13
       2024-04-15      81.34
       2024-04-12      82.69
       2024-04-11      83.57
       2024-04-10      83.48

[385536 rows x 1 columns]


In [9]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [10]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A      3.407661    9.786205  1.871824  -1.224217   -2.050601  0.814114   -3.991252    -5.309496   0.514326   -3.594632    -4.706455   0.517267   -3.312679    -4.259225   0.550503   -3.435796    -4.303416   0.534444    -1.757284     -2.360271    0.732819    -1.030629     -1.371201    0.832273
AA     3.117755    8.633473  1.769131  -1.375516   -2.437089  0.792663   -4.271969    -5.711153   0.473221   -5.018810    -6.378823   0.391698   -2.434243    -3.230505   0.651285   -2.082238    -2.707788   0.690178    -1.686742     -2.208435    0.738262    -0.466215     -0.651675    0.924103
AAL    6.144080   27.147641  3.418504   5.671178   21.628750  3.526951    0.5449

In [11]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sharpe 3d
  -Inf: ['BSCP']
Column: Sortino 3d
  Inf: ['ACGL', 'AEM', 'AFG', 'AIG', 'AJG', 'ALL', 'ANET', 'ANSS', 'APP', 'ATO', 'AVDV', 'AVGO', 'AXON', 'AXS', 'AZO', 'BA', 'BBJP', 'BINC', 'BKLN', 'BNS', 'BOKF', 'BOXX', 'BRK-A', 'BRK-B', 'BTI', 'BWXT', 'CACC', 'CB', 'CCL', 'CEG', 'CHT', 'CINF', 'CNA', 'CNC', 'CNP', 'COKE', 'COST', 'CPNG', 'CPRT', 'CRWD', 'CVNA', 'CVS', 'CW', 'CYBR', 'DB', 'DFIC', 'DRS', 'DTM', 'EEMV', 'EFAV', 'EG', 'EHC', 'ELV', 'EME', 'EQT', 'ESLT', 'ESNT', 'EVRG', 'EWJ', 'FHN', 'FIX', 'FMX', 'FNF', 'FPE', 'FTNT', 'FWONA', 'FWONK', 'FYBR', 'GD', 'GDX', 'GE', 'GEV', 'GFL', 'GGAL', 'GLD', 'GLDM', 'GRAB', 'GSIE', 'HCA', 'HDB', 'HEI', 'HEI-A', 'HIG', 'HII', 'HLN', 'HMY', 'HRB', 'HUBB', 'HUM', 'IAU', 'IBKR', 'ICL', 'INDA', 'IOT', 'ITA', 'JAAA', 'JPIE', 'JPM', 'K', 'KBR', 'KO', 'L', 'LDOS', 'LEGN', 'LHX', 'LLY', 'LMT', 'LNG', 'MELI', 'MFG', 

In [12]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'ACGL'

#### Use Cell below to check the inf, -inf and NaN replacement

In [13]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       20.192294
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       -2.851015
Sortino 5d      -3.840788
Omega 5d         0.623326
Sharpe 10d      -2.326405
Sortino 10d     -2.906123
Omega 10d        0.639846
Sharpe 15d      -0.194942
Sortino 15d     -0.248439
Omega 15d        0.962873
Sharpe 30d      -0.066473
Sortino 30d     -0.086379
Omega 30d        0.986906
Sharpe 60d      -0.038382
Sortino 60d     -0.052320
Omega 60d        0.992994
Sharpe 120d     -0.799869
Sortino 120d    -1.063844
Omega 120d       0.863346
Sharpe 250d      0.178735
Sortino 250d     0.243226
Omega 250d       1.032110
Name: ACGL, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d         20.192294
Sortino 3d      4653.430179
Omega 3d         415.560510
Sharpe 5d         -2.851015
Sortino 5d        -3.840788
Omega 5d           0.623326
Sharpe 10d        -2.326405
Sortino 10d       -2.906123
Omega 10d          0.639846
Sharpe 15d        -0.194942
Sortino 15d       -0.248439
Omega 15d          0.962873
Sharpe 30d        -0.066473
Sortino 30d       -0.086379
Omega 30d          0.986906
Sharpe 60d        -0.038382
Sortino 60d       -0.052320
Omega 60d          0.992994
Sharpe 120d       -0.799869
Sortino 120d      -1.063844
Omega 120d         0.863346
Sharpe 250d        0.178735
Sortino 250d       0.243226
Omega 250d         1.032110
Name: ACGL, dtype: float64

In [14]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-09_df_perf_ratios.parquet


In [15]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====